In [165]:
# import the libraries
import time
import re
import random as rd
import pandas as pd
import numpy as np
import collections
from tqdm import tqdm
import pickle
import datetime
from dateutil.relativedelta import relativedelta

import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

from newspaper import Article

In [166]:
# Driver setting
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
headers = {
    "User-Agent": user_agent
}

def setWebdriver():
    service = ChromeService(executable_path=ChromeDriverManager().install())    # 크롬 드라이버 최신 버전 설정

    options = ChromeOptions()
    options.add_argument('user-agent=' + user_agent)
    options.add_argument('--start-maximized') #브라우저가 최대화된 상태로 실행됩니다.
    # options.add_argument('headless') #headless모드 브라우저가 뜨지 않고 실행됩니다.
    #options.add_argument('--window-size= x, y') #실행되는 브라우저 크기를 지정할 수 있습니다.
    #options.add_argument('--start-fullscreen') #브라우저가 풀스크린 모드(F11)로 실행됩니다.
    #options.add_argument('--blink-settings=imagesEnabled=false') #브라우저에서 이미지 로딩을 하지 않습니다.
    options.add_argument('--mute-audio') #브라우저에 음소거 옵션을 적용합니다.
    options.add_argument('incognito') #시크릿 모드의 브라우저가 실행됩니다.
    driver = webdriver.Chrome(service=service, options=options)

    return driver

In [171]:
# 원하는 횟수만큼 Scroll down

def scrollDown(driver, num: int): 
    
    for _ in tqdm(range(0, num), desc='Scrolling Down...', leave='False'):
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(rd.uniform(0.1, 0.3))    

    return

In [169]:
# start_date부터 end_date까지 3개월 단위로 나누기

def getPeriods(start_date, end_date):
    
    periods = []
    while (start_date < end_date):
        if (start_date + relativedelta(months=3) > end_date):
            periods.append([start_date.strftime("%Y.%m.%d"), end_date.strftime("%Y.%m.%d")])

        else:
            periods.append([start_date.strftime("%Y.%m.%d"), (start_date + relativedelta(months=3)).strftime("%Y.%m.%d")])
        start_date += relativedelta(months=3)
    # print(periods)

    return periods

In [176]:
# 언론사 세팅

def setCompany():

    company_names = ["YTN", "JTBC", "MBC", "SBS", "국민일보", "매일경제", "조선일보", "중앙일보", "아시아경제", "한국경제", "KBS", "한겨례", "경향신문"]
    company_codes = {
        "YTN": "1052",
        "JTBC": "1437",
        "MBC": "1214",
        "SBS": "1055",
        "국민일보": "1005",
        "매일경제": "1009",
        "조선일보": "1023",
        "중앙일보": "1025",
        "아시아경제": "1277",
        "한국경제": "1015",
        "KBS": "1056",
        "한겨례": "1028",
        "경향신문": "1032",
    }

    return company_names, company_codes

In [173]:
# 기간 세팅 (3개월 단위로 나누기)

start_date = datetime.datetime.strptime("2020.09.01", "%Y.%m.%d")
end_date = datetime.datetime.strptime("2023.12.31", "%Y.%m.%d")
periods = getPeriods(start_date, end_date)

periods[0][0] = "2020.09.02"
print(periods)

[['2020.09.02', '2020.12.01'], ['2020.12.01', '2021.03.01'], ['2021.03.01', '2021.06.01'], ['2021.06.01', '2021.09.01'], ['2021.09.01', '2021.12.01'], ['2021.12.01', '2022.03.01'], ['2022.03.01', '2022.06.01'], ['2022.06.01', '2022.09.01'], ['2022.09.01', '2022.12.01'], ['2022.12.01', '2023.03.01'], ['2023.03.01', '2023.06.01'], ['2023.06.01', '2023.09.01'], ['2023.09.01', '2023.12.01'], ['2023.12.01', '2023.12.31']]


In [197]:
driver = setWebdriver()

In [136]:
company_code = companies[companies_name[0]]
query = querys[0]
start_date = periods[0][0]    # 시작 날짜
end_date = periods[0][1]    # 끝 날짜

In [137]:
url = f'https://search.naver.com/search.naver?where=news&query={query}&sm=tab_clk.jou&sort=0&photo=0&field=0&pd=3&ds={start_date}&de={end_date}&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked={company_code}&nso=so%3Ar%2Cp%3Afrom20210520to20240520&is_sug_officeid=0&office_category=0&service_area=0'
print(url)

https://search.naver.com/search.naver?where=news&query=국민의힘&sm=tab_clk.jou&sort=0&photo=0&field=0&pd=3&ds=2020.09.02&de=2020.12.01&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1052&nso=so%3Ar%2Cp%3Afrom20210520to20240520&is_sug_officeid=0&office_category=0&service_area=0


In [198]:
driver.get(url)

In [199]:
# 아래로 스크롤 할 수 있을 만큼 스크롤 다운
scrollDown(driver, 99)

Scrolling Down...: 100%|██████████| 99/99 [00:21<00:00,  4.66it/s]


In [200]:
news_list = driver.find_elements(By.CLASS_NAME, "news_tit")

In [201]:
print(len(news_list))

1000


In [203]:
news_url = news_list[0].get_attribute('href')
print(news_url)

https://www.ytn.co.kr/_ln/0101_202011301050011782


In [204]:
article = Article(news_url, language="ko")

In [ ]:
current_url = news_url
article = Article(current_url, language="ko")
article.download()
article.parse()

date = article.publish_date;    print(f'date: {date}')
authors = article.authors;      print(f'authors: {authors}')
title = article.title;  print(f'title: {title}')
text = article.text;    print(f'text: {text}')
print(f'{"-"*200}')

# dates.append(date)
# titles.append(title)
# texts.append(text)

In [191]:
def main():
    
    # 언론사 세팅
    company_names, comapny_codes = setCompany()

    # 기간 세팅 (3개월 단위로 나누기)
    start_date = datetime.datetime.strptime("2020.09.01", "%Y.%m.%d")
    end_date = datetime.datetime.strptime("2023.12.31", "%Y.%m.%d")
    periods = getPeriods(start_date, end_date)

    # 검색어 세팅
    querys = ["국민의힘", "더불어민주당"]
    
    
    driver = setWebdriver()

    for comany in tqdm(company_names, desc="Companies..."):
        company_code = comapny_codes[comany]

        for period in periods:
            start_date = period[0]
            end_date = period[1]

            for query in querys:
                url = f'https://search.naver.com/search.naver?where=news&query={query}&sm=tab_clk.jou&sort=0&photo=0&field=0&pd=3&ds={start_date}&de={end_date}&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked={company_code}&nso=so%3Ar%2Cp%3Afrom20210520to20240520&is_sug_officeid=0&office_category=0&service_area=0'

                driver.get(url)
                time.sleep(1)

                scrollDown(driver, 99)  # 아래로 스크롤 할 수 있을 만큼 스크롤 다운

                news_list = driver.find_elements(By.CLASS_NAME, "news_tit")

                for news in news_list:
                    news_url = news.get_attribute('herf')

                    driver.execute_script(f"window.open('{news_url}');")     # 새 탭 열기
                    driver.switch_to.window(driver.window_handles[1])       # 새 탭으로 이동

                    article = Article(current_url, language="ko")
                    article.download()
                    article.parse()

                    date = article.publish_date;    print(f'date: {date}')
                    authors = article.authors;      print(f'authors: {authors}')
                    title = article.title;  print(f'title: {title}')
                    text = article.text;    print(f'text: {text}')
                    print(f'{"-"*200}')
                    
                    time.sleep(1)
                    driver.close()                                          # 현재 탭 닫기
                    driver.switch_to.window(driver.window_handles[0])       # 0번째 탭으로 이동

In [ ]:
if __name__ == "__main__":
    main()

In [195]:
# 첫 번째 탭으로 이동
driver.switch_to.window(driver.window_handles[0])

news_list = driver.find_elements(By.CLASS_NAME, 'news_tit')
print(len(news_list))

0


In [206]:
df = pd.DataFrame(columns=["Date", "Title", "Text"])
df

,Date,Title,Text


In [13]:
df = pd.read_pickle('../data/JoSeonIlBo.pkl')
df

,Query,Title,Text,News
0,국민의힘,"안철수, 국민의힘 1인 시위현장 방문 “힘 보태겠다”",NaN,조선일보
1,국민의힘,국민의힘 “야당 안만나고 축구나 하는 최재성 해임하라”,NaN,조선일보
2,국민의힘,국민 60%가 “추미애·윤석열 국정조사 필요”,NaN,조선일보
3,국민의힘,"국민의힘 “親조국 삼총사, 법사위에 뭉쳤다. 활약상 상상돼”",NaN,조선일보
4,국민의힘,"野의원 “국정원, 간첩 대신 국민 부동산 등 사생활 캘수도”",NaN,조선일보
...,...,...,...,...
275,더불어민주당,"윤재옥 “민주당 총선 인재 박선원, 천안함 결함설 주장”",NaN,조선일보
276,더불어민주당,민주당 “독도 영토분쟁? 尹이 승인했는지 국정조사하자”,NaN,조선일보
277,더불어민주당,"與 “송영길 내외 부창부수…민주당, 불법행위 드러나면 검찰 탄압 주장”",NaN,조선일보
278,더불어민주당,이재명 “한동훈 취임 축하… 민주당도 협조할 것은 하겠다”,NaN,조선일보


In [ ]:
print(m.parse("국민의힘"))

국민	NNG,*,T,국민,*,*,*,*
의	JKG,*,F,의,*,*,*,*
힘	NNG,*,T,힘,*,*,*,*
EOS



In [ ]:
def get_tokenizer(tokenizer_name):
    if tokenizer_name == "komoran":
        tokenizer = Komoran()
    elif tokenizer_name == "okt":
        tokenizer = Okt()
    elif tokenizer_name == "MeCab":
        tokenizer = MeCab.Tagger()
    elif tokenizer_name == "hannanum":
        tokenizer = Hannanum()
    elif tokenizer_name == "kkma":
        tokenizer = Kkma()
    else:
        tokenizer = MeCab.Tagger()
    return tokenizer

In [ ]:
# 토크나이저별로 토큰화 결과, 시간 측정

def compareTokenizer(sentence: str, tokenizer_list: list):

    result_list = []

    for i in tokenizer_list:
        start = time.time()                                 # 시작 시간 저장
        tokenizer = get_tokenizer(i)                         # 토크나이저 불러오기

        if i == "MeCab":
            tokenized_sentence = m.parse(sentence)
        else: 
            tokenized_sentence = tokenizer.morphs(sentence)          # 토큰화

        time_required = time.time() - start                          # 종료 시간에서 시작 시간을 빼서 걸린 시간 계산
        tokenizer_and_time = i, time_required, tokenized_sentence   # 토크나이저 이름, 걸린 시간, 결과를 리스트에 저장
        
        result_list.append(tokenizer_and_time)              

    result_df = pd.DataFrame(result_list, columns=["Tokenizer", "Time", "Result"])  

    return sentence, result_df

In [ ]:
from IPython.display import display # display 함수를 이용하여 DataFrame 출력
pd.set_option('display.max_colwidth', None) # DataFrame 출력 컬럼 길이 설정

tokenizer_list = ["komoran", "okt", "MeCab", "hannanum", "kkma"]

for sentence in sentence_arr:
    
    text, result = compareTokenizer(sentence, tokenizer_list)
    print(f"{'-'*75} 본문: {text} {'-'*75}")
    display(result)

--------------------------------------------------------------------------- 본문: 김종인 제안 한달만에 성사 ---------------------------------------------------------------------------


,Tokenizer,Time,Result
0,komoran,2.206613,"[김종인, 제안, 한, 달마, ㄴ, 에, 성사]"
1,okt,0.928779,"[김종인, 제안, 한, 달, 만에, 성사]"
2,MeCab,0.001000,"김종인\tNNP,인명,T,김종인,*,*,*,*\n제안\tNNG,행위,T,제안,*,*,*,*\n한달\tXSV+ETM,*,T,한달,Inflect,XSV,ETM,하/XSV/*+ᆫ달/ETM/*\n만\tNNB,*,T,만,*,*,*,*\n에\tJKB,*,F,에,*,*,*,*\n성사\tNNG,행위,F,성사,*,*,*,*\nEOS\n"
3,hannanum,0.298229,"[김종, 이, ㄴ, 제안, 한달, 만에, 성사]"
4,kkma,2.510674,"[김, 종, 이, ㄴ, 제안, 한, 달, 만, 에, 성사]"


--------------------------------------------------------------------------- 본문: 양측 노동개혁 제휴여부 주목 ---------------------------------------------------------------------------


,Tokenizer,Time,Result
0,komoran,1.232123,"[양측, 노동, 개혁, 제휴, 여부, 주목]"
1,okt,0.006119,"[양, 측, 노동, 개혁, 제휴, 여부, 주목]"
2,MeCab,0.001000,"양측\tNNG,*,T,양측,*,*,*,*\n노동\tNNG,행위,T,노동,*,*,*,*\n개혁\tNNG,행위,T,개혁,*,*,*,*\n제휴\tNNG,행위,F,제휴,*,*,*,*\n여부\tNNG,정적사태,F,여부,*,*,*,*\n주목\tNNG,행위,T,주목,*,*,*,*\nEOS\n"
3,hannanum,0.203372,"[양측, 노동개혁, 제휴여부, 주목]"
4,kkma,0.006500,"[양측, 노동, 개혁, 제휴, 여부, 주목]"


--------------------------------------------------------------------------- 본문: 국민의힘이 노동개혁 추진을 위해 발족시킬 노동혁신특별위원회에 한국노총이 참여한다 ---------------------------------------------------------------------------


...

,Tokenizer,Time,Result
0,komoran,0.829769,"[당, 환경, 노동, 위원회, 소속, 의원, 들, 과, 기획, 재정, 위원회, 소속, 추경호, 의원, 등, 도, 참여, 하, ㄹ, 것, 으로, 전하, 아, 지, 었, 다, ., 우선, 주요, 현안, 이, ㄴ, 국제노동기구, (, ILO, ), 핵심, 협약, 비준, ,, 특수, 고용, 노동자, 권익, 보호, ,, 중대, 재해, 기업, 처벌, 법, 제정, ,, 근로, 시간, 유연, 화, 등, 의, 현안, 이, 특위, 에서, 논의, 되, ㄹ, 방침, 이, 다]"
1,okt,0.010015,"[당, 환경, 노동, 위원회, 소속, 의원, 들, 과, 기획, 재정, 위원회, 소속, 추경호, 의원, 등, 도, 참여, 할, 것, 으로, 전해졌다, ., 우선, 주요, 현안, 인, 국제노동기구, (, ILO, ), 핵심, 협약, 비준, ,, 특수, 고용, 노동자, 권익, 보호, ,, 중대, 재해, 기업, 처벌, 법, 제정, ,, 근로시간, 유연, 화, 등, 의, 현안, 이, 특위, 에서, 논의, 될, 방침, 이다]"
2,MeCab,0.001000,"당\tNNG,*,T,당,*,*,*,*\n환경\tNNG,*,T,환경,*,*,*,*\n노동\tNNG,행위,T,노동,*,*,*,*\n위원회\tNNG,*,F,위원회,Compound,*,*,위원/NNG/*+회/NNG/*\n소속\tNNG,*,T,소속,*,*,*,*\n의원\tNNG,*,T,의원,*,*,*,*\n들\tXSN,*,T,들,*,*,*,*\n과\tJKB,*,F,과,*,*,*,*\n기획\tNNG,행위,T,기획,*,*,*,*\n재정\tNNG,*,T,재정,*,*,*,*\n위원회\tNNG,*,F,위원회,Compound,*,*,위원/NNG/*+회/NNG/*\n소속\tNNG,*,T,소속,*,*,*,*\n추경호\tNNP,인명,F,추경호,*,*,*,*\n의원\tNNG,*,T,의원,*,*,*,*\n등\tNNB,*,T,등,*,*,*,*\n도\tJX,*,F,도,*,*,*,*\n참여\tNNG,행위,F,참여,*,*,*,*\n할\tXSV+ETM,*,T,할,Inflect,XSV,ETM,하/XSV/*+ᆯ/ETM/*\n것\tNNB,*,T,것,*,*,*,*\n으로\tJKB,*,F,으로,*,*,*,*\n전해졌\tVV+EC+VX+EP,*,T,전해졌,Inflect,VV,EP,전하/VV/*+아/EC/*+지/VX/*+었/EP/*\n다\tEF,*,F,다,*,*,*,*\n.\tSF,*,*,*,*,*,*,*\n우선\tMAG,성분부사|시간부사,T,우선,*,*,*,*\n주요\tNNG,*,F,주요,*,*,*,*\n현안\tNNG,*,T,현안,*,*,*,*\n인\tVCP+ETM,*,T,인,Inflect,VCP,ETM,이/VCP/*+ᆫ/ETM/*\n국제노동기구\tNNP,*,F,국제노동기구,Compound,*,*,국제/NNG/*+노동/NNG/*+기구/NNG/*\n(\tSSO,*,*,*,*,*,*,*\nILO\tSL,*,*,*,*,*,*,*\n)\tSSC,*,*,*,*,*,*,*\n핵심\tNNG,*,T,핵심,*,*,*,*\n협약\tNNG,행위,T,협약,*,*,*,*\n비준\tNNG,행위,T,비준,*,*,*,*\n,\tSC,*,*,*,*,*,*,*\n특수\tNNG,*,F,특수,*,*,*,*\n고용\tNNG,행위,T,고용,*,*,*,*\n노동자\tNNG,*,F,노동자,Compound,*,*,노동/NNG/*+자/NNG/*\n권익\tNNG,*,T,권익,*,*,*,*\n보호\tNNG,행위,F,보호,*,*,*,*\n,\tSC,*,*,*,*,*,*,*\n중대\tNNG,*,F,중대,*,*,*,*\n재해\tNNG,*,F,재해,*,*,*,*\n기업\tNNG,*,T,기업,*,*,*,*\n처벌\tNNG,행위,T,처벌,*,*,*,*\n법\tNNG,*,T,법,*,*,*,*\n제정\tNNG,*,T,제정,*,*,*,*\n,\tSC,*,*,*,*,*,*,*\n근로\tNNG,행위,F,근로,*,*,*,*\n시간\tNNG,*,T,시간,*,*,*,*\n유연\tXR,*,T,유연,*,*,*,*\n화\tXSN,*,F,화,*,*,*,*\n등\tNNB,*,T,등,*,*,*,*\n의\tJKG,*,F,의,*,*,*,*\n현안\tNNG,*,T,현안,*,*,*,*\n이\tJKS,*,F,이,*,*,*,*\n특위\tNNG,*,F,특위,*,*,*,*\n에서\tJKB,*,F,에서,*,*,*,*\n논의\tNNG,*,F,논의,*,*,*,*\n될\tXSV+ETM,*,T,될,Inflect,XSV,ETM,되/XSV/*+ᆯ/ETM/*\n방침\tNNG,*,T,방침,*,*,*,*\n이\tVCP,*,F,이,*,*,*,*\n다\tEC,*,F,다,*,*,*,*\nEOS\n"
3,hannanum,0.162899,"[당, 환경노동위원회, 소속, 의원들, 과, 기획재정위원회, 소속, 추경호, 의원, 등, 도, 참여, 하, ㄹ, 것, 으로, 전하, 어, 지, 었다, ., 우선, 주요, 현안, 이, ㄴ, 국제노동기구(ILO), 핵심, 협약, 비준, ,, 특수고용노동자, 권익, 보호, ,, 중대재해기업처벌법, 제정, ,, 근로시간, 유연화, 등, 의, 현안, 이, 특위, 에서, 논의, 되, ㄹ, 방침, 이, 다]"
4,kkma,0.046801,"[당, 환경, 노동, 위원회, 소속, 의원, 들, 과, 기획, 재정, 위원회, 소속, 추경, 호, 의원, 등, 도, 참여, 하, ㄹ, 것, 으로, 전하, 어, 지, 었, 다, ., 우선, 주요, 현안, 이, ㄴ, 국제, 노동, 기구, (, ILO, ), 핵심, 협약, 비준, ,, 특수, 고용, 노동자, 권익, 보호, ,, 중대, 재해, 기업, 처벌법, 제정, ,, 근로, 시간, 유, 연화, 등, 의, 현안, 이, 특위, 에서, 논의, 되, ㄹ, 방침, 이, 다]"


--------------------------------------------------------------------------- 본문: 이 중에서도 김종인 위원장이 공개 제안한 노동관계법 개정을 통한 노동시장 유연화 정책은 노조에서 반발하는 내용이라 특위에서 논의될 경우 합의점 도출이 쉽지 않을 것이란 전망이 나온다.이에 대해 임 의원은 "특위 출범은 일방적으로 쟁점 법안을 통과시키기보다는 노사 각자의 입장을 듣자는 데 취지가 있다"며 "조금이라도 양보가 얻어져 합의가 이뤄지면 금상첨화겠지만, 합의가 안 되더라도 특위 참여 의원들이 양측 입장을 반영하기 위해 노력하겠다"고 말했다 ---------------------------------------------------------------------------


,Tokenizer,Time,Result
0,komoran,0.800086,"[이, 중, 에서, 도, 김종인, 위원장, 이, 공개, 제안, 하, ㄴ, 노동관계, 법, 개정, 을, 통하, ㄴ, 노동, 시장, 유연, 화, 정책, 은, 노조, 에서, 반발, 하, 는, 내용, 이, 라, 특위, 에서, 논의, 되, ㄹ, 경우, 합의점, 도출, 이, 쉽, 지, 않, 을, 것, 이란, 전망, 이, 나오, ㄴ다, ., 이, 에, 대하, 아, 임, 의원, 은, "", 특위, 출범, 은, 일방, 적, 으로, 쟁점, 법안, 을, 통과, 시키, 기, 보다, 는, 노사, 각자, 의, 입장, 을, 듣, 자는, 데, 취지, 가, 있다, "", 이, 며, "", 조금, 이, 라도, 양보, 가, 얻, 어, 지, 어, 합의, 가, 이루어지, ...]"
1,okt,0.013009,"[이, 중, 에서도, 김종인, 위원장, 이, 공개, 제안, 한, 노동, 관계, 법, 개정, 을, 통한, 노동시장, 유연, 화, 정책, 은, 노조, 에서, 반발, 하는, 내용, 이라, 특위, 에서, 논의, 될, 경우, 합의, 점, 도출, 이, 쉽지, 않을, 것, 이란, 전망, 이, 나온다, ., 이, 에, 대해, 임, 의원, 은, "", 특위, 출범, 은, 일방, 적, 으로, 쟁점, 법안, 을, 통과, 시키, 기, 보다는, 노사, 각자, 의, 입장, 을, 듣자는, 데, 취지, 가, 있다, "", 며, "", 조금, 이라도, 양보, 가, 얻어져, 합의, 가, 이뤄지면, 금상첨화, 겠지만, ,, 합의, 가, 안, 되더라도, 특위, 참여, 의원, 들, 이, 양, 측, 입장, 을, ...]"
2,MeCab,0.001000,"이\tMM,~명사,F,이,*,*,*,*\n중\tNNB,*,T,중,*,*,*,*\n에서\tJKB,*,F,에서,*,*,*,*\n도\tJX,*,F,도,*,*,*,*\n김종인\tNNP,인명,T,김종인,*,*,*,*\n위원장\tNNG,*,T,위원장,Compound,*,*,위원/NNG/*+장/NNG/*\n이\tJKS,*,F,이,*,*,*,*\n공개\tNNG,행위,F,공개,*,*,*,*\n제안\tNNG,행위,T,제안,*,*,*,*\n한\tXSV+ETM,*,T,한,Inflect,XSV,ETM,하/XSV/*+ᆫ/ETM/*\n노동관계\tNNG,*,F,노동관계,Compound,*,*,노동/NNG/*+관계/NNG/*\n법\tNNG,*,T,법,*,*,*,*\n개정\tNNG,행위,T,개정,*,*,*,*\n을\tJKO,*,T,을,*,*,*,*\n통한\tVV+ETM,*,T,통한,Inflect,VV,ETM,통하/VV/*+ᆫ/ETM/*\n노동\tNNG,행위,T,노동,*,*,*,*\n시장\tNNG,*,T,시장,*,*,*,*\n유연\tXR,*,T,유연,*,*,*,*\n화\tXSN,*,F,화,*,*,*,*\n정책\tNNG,*,T,정책,*,*,*,*\n은\tJX,*,T,은,*,*,*,*\n노조\tNNG,*,F,노조,*,*,*,*\n에서\tJKB,*,F,에서,*,*,*,*\n반발\tNNG,행위,T,반발,*,*,*,*\n하\tXSV,*,F,하,*,*,*,*\n는\tETM,*,T,는,*,*,*,*\n내용\tNNG,*,T,내용,*,*,*,*\n이\tVCP,*,F,이,*,*,*,*\n라\tEC,*,F,라,*,*,*,*\n특위\tNNG,*,F,특위,*,*,*,*\n에서\tJKB,*,F,에서,*,*,*,*\n논의\tNNG,*,F,논의,*,*,*,*\n될\tXSV+ETM,*,T,될,Inflect,XSV,ETM,되/XSV/*+ᆯ/ETM/*\n경우\tNNG,*,F,경우,*,*,*,*\n합의\tNNG,행위,F,합의,*,*,*,*\n점\tNNG,*,T,점,*,*,*,*\n도출\tNNG,행위,T,도출,*,*,*,*\n이\tJKS,*,F,이,*,*,*,*\n쉽\tVA,*,T,쉽,*,*,*,*\n지\tEC,*,F,지,*,*,*,*\n않\tVX,*,T,않,*,*,*,*\n을\tETM,*,T,을,*,*,*,*\n것\tNNB,*,T,것,*,*,*,*\n이\tVCP,*,F,이,*,*,*,*\n란\tETM,*,T,란,*,*,*,*\n전망\tNNG,*,T,전망,*,*,*,*\n이\tJKS,*,F,이,*,*,*,*\n나온다\tVV+EF,*,F,나온다,Inflect,VV,EF,나오/VV/*+ᆫ다/EF/*\n.\tSF,*,*,*,*,*,*,*\n이\tNP,*,F,이,*,*,*,*\n에\tJKB,*,F,에,*,*,*,*\n대해\tVV+EC,*,F,대해,Inflect,VV,EC,대하/VV/*+아/EC/*\n임\tNNP,인명,T,임,*,*,*,*\n의원\tNNG,*,T,의원,*,*,*,*\n은\tJX,*,T,은,*,*,*,*\n""\tSY,*,*,*,*,*,*,*\n특위\tNNG,*,F,특위,*,*,*,*\n출범\tNNG,행위,T,출범,*,*,*,*\n은\tJX,*,T,은,*,*,*,*\n일방\tNNG,*,T,일방,*,*,*,*\n적\tXSN,*,T,적,*,*,*,*\n으로\tJKB,*,F,으로,*,*,*,*\n쟁점\tNNG,*,T,쟁점,*,*,*,*\n법안\tNNG,*,T,법안,*,*,*,*\n을\tJKO,*,T,을,*,*,*,*\n통과\tNNG,행위,F,통과,*,*,*,*\n시키\tXSV,*,F,시키,*,*,*,*\n기\tETN,*,F,기,*,*,*,*\n보다\tJKB,*,F,보다,*,*,*,*\n는\tJX,*,T,는,*,*,*,*\n노사\tNNG,*,F,노사,*,*,*,*\n각자\tNNG,*,F,각자,*,*,*,*\n의\tJKG,*,F,의,*,*,*,*\n입장\tNNG,*,T,입장,*,*,*,*\n을\tJKO,*,T,을,*,*,*,*\n듣\tVV,*,T,듣,Inflect,VV,VV,들/VV/*\n자는\tETM,*,T,자는,*,*,*,*\n데\tNNB,*,F,데,*,*,*,*\n취지\tNNG,*,F,취지,*,*,*,*\n가\tJKS,*,F,가,*,*,*,*\n있\tVA,*,T,있,*,*,*,*\n다\tEC,*,F,다,*,*,*,*\n""\tSY,*,*,*,*,*,*,*\n며\tEC,*,F,며,*,*,*,*\n""\tSY,*,*,*,*,*,*,*\n조금\tNNG,*,T,조금,*,*,*,*\n이\tVCP,*,F,이,*,*,*,*\n라도\tEC,*,F,라도,*,*,*,*\n양보\tNNG,행위,F,양보,*,*,*,*\n가\tJKS,*,F,가,*,*,*,*\n얻\tVV,*,T,얻,*,*,*,*\n어\tEC,*,F,어,*,*,*,*\n져\tVX+EC,*,F,져,Inflect,VX,EC,지/VX/*+어/EC/*\n합의\tNNG,행위,F,합의,*,*,*,*\n가\tJKS,*,F,가,*,*,*,*\n이뤄지면\tVV+EC,*,T,이뤄지면,Inflect,VV,EC,이루어지/VV/*+면/EC/*\n금상첨화\tNNG,*,F,금상첨화,*,*,*,*\n겠\tVCP+EP,*,T,겠,Inflect,VCP,EP,이/VCP/*+겠/EP/*\n지만\tEC,*,T,지만,*,*,*,*\n,\tSC,*,*,*,*,*,*,*\n합의\tNNG,행위,F,합의,*,*,*,*\n가\tJKS,*,F,가,*,*,*,*\n안\tMAG,성분부사|부정부사,T,안,*,*,*,*\n되\tVV,*,F,되,*,*,*,*\n더라도\tEC,*,F,더라도,*,*,*,*\n특위\tNNG,*,F,특위,*,*,*,*\n참여\tNNG,행위,F,참여,*,*,*,*\n의원\tNNG,*,T,의원,*,*,*,*\n들\tXSN,*,T,들,*,*,*,*\n이\tJKS,*,F,이,*,*,*,*\n양측\tNNG,*,T,양측,*,*,*,*\n입장\tNNG,*,T,입장,*,*,*,*\n을\tJKO,*,T,을,*,*,*,*\n반영\tNNG,*,T,반영,*,*,*,*\n하\tXSV,*,F,하,*,*,*,*\n기\tETN,*,F,기,*,*,*,*\n위해\tVV+EC,*,F,위해,Inflect,VV,EC,위하/VV/*+아/EC/*\n노력\tNNG,*,T,노력,*,*,*,*\n하\tXSV,*,F,하,*,*,*,*\n겠\tEP,*,T,겠,*,*,*,*\n다\tEC,*,F,다,*,*,*,*\n""\tSY,*,*,*,*,*,*,*\n고\tJKQ,*,F,고,*,*,*,*\n말\tNNG,*,T,말,*,*,*,*\n했\tXSV+EP,*,T,했,Inflect,XSV,

--------------------------------------------------------------------------- 본문: 이에 한 국민의힘 환노위 관계자는 "노조 측에서도 여야에 목소리가 골고루 전달돼야 법안이 만들어질 수 있다는 걸 아니까 특위에 참여해 입장을 밝히려는 것으로 보인다"고 말했다.[정주원 기자][ⓒ 매일경제 & mk.co.kr, 무단전재 및 재배포 금지] ---------------------------------------------------------------------------


,Tokenizer,Time,Result
0,komoran,0.813108,"[이, 에, 한, 국민, 의, 힘, 환, 노, 위, 관계자, 는, "", 노조, 측, 에서, 도, 여야, 에, 목소리, 가, 골고루, 전달, 되, 어야, 법안, 이, 만들, 어, 지, ㄹ, 수, 있, 다는, 걸, ㄹ, 알, 니까, 특위, 에, 참여, 하, 아, 입장, 을, 밝히, 려는, 것, 으로, 보이, ㄴ다, "", 고, 말, 하, 았, 다, ., [, 정주원, 기자, ], [, ⓒ, 매일, 경제, &, mk, .co, .kr, ,, 무단, 전재, 및, 재, 배포, 금지, ]]"
1,okt,0.004000,"[이, 에, 한, 국민, 의, 힘, 환노위, 관계자, 는, "", 노조, 측, 에서도, 여야, 에, 목소리, 가, 골고루, 전달, 돼야, 법안, 이, 만들어질, 수, 있다는, 걸, 아니까, 특위, 에, 참여, 해, 입장, 을, 밝히려는, 것, 으로, 보인다, "", 고, 말, 했다, .[, 정주, 원, 기자, ][, ⓒ, 매일경제, &, mk.co.kr, ,, 무단, 전, 재, 및, 재, 배포, 금지, ]]"
2,MeCab,0.001501,"이\tNP,*,F,이,*,*,*,*\n에\tJKB,*,F,에,*,*,*,*\n한\tMM,~가산명사,T,한,*,*,*,*\n국민\tNNG,*,T,국민,*,*,*,*\n의\tJKG,*,F,의,*,*,*,*\n힘\tNNG,*,T,힘,*,*,*,*\n환노위\tNNP,*,F,환노위,Compound,*,*,환/NNG/*+노/NNG/*+위/NNG/*\n관계자\tNNG,*,F,관계자,Compound,*,*,관계/NNG/*+자/NNG/*\n는\tJX,*,T,는,*,*,*,*\n""\tSY,*,*,*,*,*,*,*\n노조\tNNG,*,F,노조,*,*,*,*\n측\tNNB,*,T,측,*,*,*,*\n에서\tJKB,*,F,에서,*,*,*,*\n도\tJX,*,F,도,*,*,*,*\n여야\tNNG,*,F,여야,*,*,*,*\n에\tJKB,*,F,에,*,*,*,*\n목소리\tNNG,*,F,목소리,Compound,*,*,목/NNG/*+소리/NNG/*\n가\tJKS,*,F,가,*,*,*,*\n골고루\tMAG,성분부사|양태부사,F,골고루,*,*,*,*\n전달\tNNG,*,T,전달,*,*,*,*\n돼야\tXSV+EC,*,F,돼야,Inflect,XSV,EC,되/XSV/*+어야/EC/*\n법안\tNNG,*,T,법안,*,*,*,*\n이\tJKS,*,F,이,*,*,*,*\n만들\tVV,*,T,만들,*,*,*,*\n어\tEC,*,F,어,*,*,*,*\n질\tVX+ETM,*,T,질,Inflect,VX,ETM,지/VX/*+ㄹ/ETM/*\n수\tNNB,*,F,수,*,*,*,*\n있\tVV,*,T,있,*,*,*,*\n다는\tETM,*,T,다는,*,*,*,*\n걸\tNNB+JKO,*,T,걸,Inflect,NNB,JKO,것/NNB/*+ᆯ/JKO/*\n아\tVV,*,F,아,Inflect,VV,VV,알/VV/*\n니까\tEC,*,F,니까,*,*,*,*\n특위\tNNG,*,F,특위,*,*,*,*\n에\tJKB,*,F,에,*,*,*,*\n참여\tNNG,행위,F,참여,*,*,*,*\n해\tXSV+EC,*,F,해,Inflect,XSV,EC,하/XSV/*+아/EC/*\n입장\tNNG,*,T,입장,*,*,*,*\n을\tJKO,*,T,을,*,*,*,*\n밝히\tVV,*,F,밝히,*,*,*,*\n려는\tETM,*,T,려는,*,*,*,*\n것\tNNB,*,T,것,*,*,*,*\n으로\tJKB,*,F,으로,*,*,*,*\n보인다\tVV+EC,*,F,보인다,Inflect,VV,EC,보이/VV/*+ᆫ다/EC/*\n""\tSY,*,*,*,*,*,*,*\n고\tJKQ,*,F,고,*,*,*,*\n말\tNNG,*,T,말,*,*,*,*\n했\tXSV+EP,*,T,했,Inflect,XSV,EP,하/XSV/*+았/EP/*\n다\tEF,*,F,다,*,*,*,*\n.\tSF,*,*,*,*,*,*,*\n[\tSSO,*,*,*,*,*,*,*\n정\tNNP,인명,T,정,*,*,*,*\n주원\tNNG,*,T,주원,*,*,*,*\n기자\tNNG,*,F,기자,*,*,*,*\n]\tSSC,*,*,*,*,*,*,*\n[\tSSO,*,*,*,*,*,*,*\nⓒ\tSY,*,*,*,*,*,*,*\n매일\tMAG,성분부사|시간부사,T,매일,*,*,*,*\n경제\tNNG,*,F,경제,*,*,*,*\n&\tSY,*,*,*,*,*,*,*\nmk\tSL,*,*,*,*,*,*,*\n.\tSY,*,*,*,*,*,*,*\nco\tSL,*,*,*,*,*,*,*\n.\tSY,*,*,*,*,*,*,*\nkr\tSL,*,*,*,*,*,*,*\n,\tSC,*,*,*,*,*,*,*\n무단\tNNG,정적사태,T,무단,*,*,*,*\n전재\tNNG,*,F,전재,*,*,*,*\n및\tMAJ,*,T,및,*,*,*,*\n재\tXPN,*,F,재,*,*,*,*\n배포\tNNG,정적사태,F,배포,*,*,*,*\n금지\tNNG,행위,F,금지,*,*,*,*\n]\tSSC,*,*,*,*,*,*,*\nEOS\n"
3,hannanum,0.146725,"[이, 에, 하, ㄴ, 국민의힘, 환노위, 관계자, 는, "", 노조, 측에서, 도, 여야, 에, 목소리, 가, 골고루, 전달, 되, 어야, 법안, 이, 만들, 어, 지, ㄹ, 수, 있, 다는, 거, ㄹ, 알, 니까, 특위, 에, 참여, 하, 어, 입장, 을, 밝히, 려는, 것, 으로, 보이, ㄴ다, "", 고, 말, 하, 었다, ., [, 정주원, 기자][ⓒ, 매일경제, &, mk, ., co, ., kr, ,, 무단전재, 및, 재배포, 금지, ]]"
4,kkma,0.043261,"[이에, 한, 국민, 의, 힘, 환, 노, 위, 관계자, 는, "", 노조, 측, 에서, 도, 여야, 에, 목, 소리, 가, 골고루, 전달, 되, 어야, 법안, 이, 만들, 어, 지, ㄹ, 수, 있, 다는, 것, 을, 알, 니까, 특위, 에, 참여, 하, 어, 입장, 을, 밝히, 려는, 것, 으로, 보이, ㄴ다, "", 고, 말하, 었, 다, ., [, 정, 주원, 기자, ], [, ⓒ, 매일, 경제, &, mk, ., co, ., kr, ,, 무단, 전재, 및, 재, 배포, 금지, ]]"


In [49]:
import pandas as pd

# 예시 DataFrame 생성
df = pd.DataFrame({
    'column1': ['A', 'B', 'A', 'B', 'C', 'A', 'B', 'C', 'C'],
    'column2': [1, 2, 3, 4, 5, 6, 7, 8, 9]
})

# 'column1'에서 중복된 값을 찾음
duplicates = df['column1'].duplicated(keep=False)

print(duplicates)

0    True
1    True
2    True
3    True
4    True
5    True
6    True
7    True
8    True
Name: column1, dtype: bool


In [55]:
df= pd.read_pickle('../data/AsiaGyeongJe.pkl')
# print(df)
# print(df["Title"].duplicated().sum())
# print(df["Text"].duplicated().sum())

# dataframe 특정 열의 중복 확인
# df = df

print(df.shape)
print(df["Text"].duplicated().sum())
# 중복된 행 출력
print(df[df["Text"].duplicated(keep=False)])

(280, 4)
17
      Query                                              Title  \
2      국민의힘           [아경 여론조사]서울 지지도, 민주당 34.9% vs 국민의힘 24.7%   
13     국민의힘             국민의힘 "민주당, 선거 앞 대통령까지 이용…원내대표 압박 적반하장"   
23     국민의힘          [아경 여론조사] 정당 지지도, 더불어민주당 31.5% 국민의힘 28.7%   
35     국민의힘  'TV토론 30분 전' 이준석 보이콧..野 "민주당 입법 강행" vs 與 "국민과 ...   
62     국민의힘   방송 3사 출구조사서 '국민의힘 10곳, 민주당 4곳' 우세…'국정 안정' 선택(상보)   
63     국민의힘                "안정론 국민의힘 vs 견제론 민주당"…정국주도권 걸린 지방선거   
68     국민의힘        MBC "6·1지방선거 광역단체장 국민의힘 8곳, 민주당 4곳에서 유력·확실"   
79     국민의힘          국민의힘, 이재명 선출 짤막한 논평…"국민을 위한, 국민의 민주당이 되길"   
101    국민의힘                       [포토] 국민의힘, 코인게이트 진상조사단 4차 회의   
109    국민의힘                 [속보]하영제 의원, 국민의힘 탈당… '정치자금법 위반' 기소   
126    국민의힘                        [포토] 원내대책회의 참석하는 국민의힘 원내지도부   
127    국민의힘             [속보]국민의힘 혁신위 '친윤·지도부 희생' 공식의결…지도부 최후통첩   
133    국민의힘                           [속보]국민의힘, 신임 사무총장 장동혁 의원   
140  더불어민주당           [아경 여론조사]서울 지지도, 민주당 34.9% vs 국민의힘 24.7%  

In [39]:
print(df.iloc[109]["Text"])

가장 많이 읽힌 뉴스를 제공합니다. 집계 기준에 따라 최대 3일 전 기사까지 제공될 수 있습니다.


In [28]:
df.isnull().sum()

Unnamed: 0     0
Query          0
Date          14
Title         14
Text          20
dtype: int64

In [5]:
import pandas as pd
tmp1 = pd.DataFrame(columns=["Query", "Date", "Title", "Text"])

In [18]:
tmp3 = pd.DataFrame({"Query": "123", "Date": "12", "Title": '123', "Text": '123'}, index=([len(tmp)]))
tmp = pd.concat([tmp, tmp3])   


In [19]:
tmp

,Query,Date,Title,Text
0,123,12,123,123
0,123,12,123,123
2,123,12,123,123
3,123,12,123,123


### Data 확인

In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
datasets = os.listdir('../data/')
print(datasets)

['AsiaGyeongJe.pkl', 'GukMinIlBo.pkl', 'GyeongHyang.pkl', 'HanGukGyeongJe.pkl', 'HanGyeoRye.pkl', 'JoSeonIlBo.pkl', 'JTBC.pkl', 'JungAngIlBo.pkl', 'KBS.pkl', 'MaeIlGyeongJe.pkl', 'MBC.pkl', 'SBS.pkl', 'YTN.pkl']


In [22]:
for dataset in datasets:
    print(f"------------------------------------------------------------------- {dataset} -------------------------------------------------------------------")
    df = pd.read_pickle(f'../data/{dataset}')
    # df = df.drop(["Unnamed: 0", "Date"], axis=1)   # df.drop 설명: 
    print(df.columns)
    print(f'shape: {df.shape}')
    print(f'Title duplicated: {df["Title"].duplicated().sum()}')
    print(f'Text duplicated: {df["Text"].duplicated().sum()}')
    # print(f'null:\n{df.isnull().sum()}')
    print(df)
    df.to_pickle(f"../data/{dataset.replace('.csv','')}")

------------------------------------------------------------------- AsiaGyeongJe.pkl -------------------------------------------------------------------
Index(['Query', 'Title', 'Text', 'News'], dtype='object')
shape: (280, 4)
Title duplicated: 8
Text duplicated: 17
      Query                                     Title  \
0      국민의힘               국민 10명 중 6명 "秋·尹 국정조사 필요하다"   
1      국민의힘               국민의힘, 혁신노동특위 구성 완료…한국노총도 참여   
2      국민의힘  [아경 여론조사]서울 지지도, 민주당 34.9% vs 국민의힘 24.7%   
3      국민의힘                        [포토] 국민의힘, 법사위 보이콧   
4      국민의힘        국민의힘, '尹 직무배제 부당' 감찰위 결정에 "상식에 부합"   
..      ...                                       ...   
275  더불어민주당             [포토] 한동훈 비대위원장, 이재명 민주당 대표 예방   
276  더불어민주당            [포토]정세균 전 총리와 회동 나선 이재명 민주당 대표   
277  더불어민주당         검찰, 민주당 부대변인 압수수색… 김용 재판 위증 관여 의혹   
278  더불어민주당           대법, '10억대 금품수수' 이정근 징역 4년2개월 확정   
279  더불어민주당         민주당 또 여성비하?…"국힘은 불임정당" 썼다 삭제한 민형배   

                                                

In [5]:
import pandas as pd

df.drop('Unnamed: 0', axis=1, inplace=True)


In [42]:
def addCompany(filename: str, company: str):
    df = pd.read_pickle(f'../data/{filename}.pkl')
    df['News'] = company
    df.to_pickle(f'../data/{filename}.pkl')
    print(df)
    
    return

In [60]:
filenames = [file.replace(".pkl", '') for file in os.listdir('../data/')]
print(filenames)
print(len(filenames))

['AsiaGyeongJe', 'GukMinIlBo', 'GyeongHyang', 'HanGukGyeongJe', 'HanGyeoRye', 'JoSeonIlBo', 'JTBC', 'JungAngIlBo', 'KBS', 'MaeIlGyeongJe', 'MBC', 'SBS', 'YTN']
13


In [59]:
companies = ["아시아경제", "국민일보", "경향신문", "한국경제", "한겨례", "조선일보", "JTBC", "중앙일보", "KBS", "매일경제", "MBC", "SBS", "YTN"]
print(len(companies))

13


In [61]:
for i in range(len(filenames)):
    addCompany(filenames[i], companies[i])


      Query                                     Title  \
0      국민의힘               국민 10명 중 6명 "秋·尹 국정조사 필요하다"   
1      국민의힘               국민의힘, 혁신노동특위 구성 완료…한국노총도 참여   
2      국민의힘  [아경 여론조사]서울 지지도, 민주당 34.9% vs 국민의힘 24.7%   
3      국민의힘                        [포토] 국민의힘, 법사위 보이콧   
4      국민의힘        국민의힘, '尹 직무배제 부당' 감찰위 결정에 "상식에 부합"   
..      ...                                       ...   
275  더불어민주당             [포토] 한동훈 비대위원장, 이재명 민주당 대표 예방   
276  더불어민주당            [포토]정세균 전 총리와 회동 나선 이재명 민주당 대표   
277  더불어민주당         검찰, 민주당 부대변인 압수수색… 김용 재판 위증 관여 의혹   
278  더불어민주당           대법, '10억대 금품수수' 이정근 징역 4년2개월 확정   
279  더불어민주당         민주당 또 여성비하?…"국힘은 불임정당" 썼다 삭제한 민형배   

                                                  Text   News  
0    [아시아경제 강나훔 기자] 국민 10명 중 6명은 추미애 법무부 장관과 윤석열 검찰...  아시아경제  
1    [아시아경제 김혜민 기자] 노동 현안을 논의할 국민의힘 노동혁신특별위원회에 한국노총...  아시아경제  
2    아시아경제 의뢰, '정당 지지도 조사' 결과\n\n전체 민주당 34.7% > 국민의...  아시아경제  
3    국민의힘 법사위원들이 30일 국회에서 열린 법사위 전체회의에 불참하고 있다. 이날 ...  아시아

In [57]:
print(len(filenames))

13


In [59]:
import pandas as pd

df = pd.read_pickle("../data/tokenized_total.pkl")
df

,Query,Title,Text,News,Tokenized
0,더불어민주당,막대기도 당선될 판 진중권 네거티브 민주당 비판,진중권 전 동양대 교수 이미지출처연합뉴스 AD 원본보기 아이콘아시아경제 황수미 기자...,asiae,"[진중권, 전, 동양, 대, 교수, 이미지, 출처, 연합뉴스, AD, 원본, 보, ..."
1,더불어민주당,1년만에 뒤바뀐 공수민주당 사죄 국민의힘 여론조사가 민심,이낙연 분노와 실망 아프도록 잘 안다 반성하고 혁신주호영 여론조사 2030 차이 민...,asiae,"[이낙연, 분노, 와, 실망, 아프, 도록, 잘, 안, 다, 반성, 하, 고, 혁신..."
2,더불어민주당,종합민주당 오세훈 내곡동 특혜 의혹에 거짓말이 거짓말을 낳아 맹공,박영선 TV 토론서 오세훈 내곡동 특혜 의혹 두고 공세47 서울시장 보궐선거 더불어...,asiae,"[박영선, TV, 토론, 서, 오세훈, 내곡동, 특혜, 의혹, 두, 고, 공세, 4..."
3,더불어민주당,박형준 안민석 더불어민주당 의원진보 유튜버 등 4명 부산지검 고발,국민의힘 캠프 부동산 투기 전혀 없다 안 의원 후보부인 부동산 복부인 발언에 반박...,asiae,"[국민의힘, 캠프, 부동산, 투기, 전혀, 없, 다, 안, 의원, 후보, 부인, 부..."
4,더불어민주당,포토 더불어민주당 원내대책회의,가장 많이 읽힌 뉴스를 제공합니다 집계 기준에 따라 최대 3일 전 기사까지 제공될 ...,asiae,"[가장, 많이, 읽힌, 뉴스, 를, 제공, 합니다, 집계, 기준, 에, 따라, 최대..."
...,...,...,...,...,...
181819,더불어민주당,여론톡톡 김기현 책임론 확산이낙연이준석 손잡나,AD 진행 김영수 앵커 출연 김형준 배재대 석좌교수 배종찬 인사이트 K 연구소장...,YTN,"[AD, 진행, 김영수, 앵커, 출연, 김형준, 배재, 대, 석좌, 교수, 배종찬,..."
181820,더불어민주당,홍익표 쌍특검 8일 본회의국정조사도 연내 처리,AD더불어민주당 홍익표 원내대표는 쌍특검 법안과 예산안은 오는 8일 본회의에 올리고...,YTN,"[AD, 더불어민주당, 홍익표, 원내대표, 는, 쌍, 특검, 법안, 과, 예산안, ..."
181821,더불어민주당,손학규 병립형 회귀 큰 후퇴이재명 약속 지켜야,AD손학규 전 바른미래당 대표는 선거제 개편 논의와 관련해 연동형 비례제는 더불어민...,YTN,"[AD, 손학규, 전, 바른, 미래, 당, 대표, 는, 선거제, 개편, 논의, 와,..."
181822,더불어민주당,민주 송영길 구속영장 청구 검찰 독재정권의 총선 전략,AD더불어민주당은 검찰이 전당대회 돈봉투 의혹에 연루된 송영길 전 대표에게 구속영장...,YTN,"[AD, 더불어민주당, 은, 검찰, 이, 전당, 대회, 돈, 봉투, 의혹, 에, 연..."
